In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

/Users/niravadunuthula/repo/cs499/CSCI499_NaturalLanguageforInteractiveAI/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch import cuda

device = "cuda:0" if cuda.is_available() else "cpu"

In [ ]:
#!pip install tensorflow==2.9

In [3]:
import os
from fastai.text.all import *

In [4]:
json_path = '/Users/niravadunuthula/repo/cs499/PoetryGen'
kaggle_dir_path = '~/.kaggle'
data_path = '/Users/niravadunuthula/repo/cs499/PoetryGen/VillanelleGen/poemsdataset'

In [10]:
# add kaggle api key to root
!cp {json_path + '/kaggle.json'} {kaggle_dir_path}
!chmod 600 {kaggle_dir_path + '/kaggle.json'}

# download poem data
!kaggle datasets download -d michaelarman/poemsdataset
!unzip -n poemsdataset.zip -d {data_path}

poemsdataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  poemsdataset.zip
error:  cannot create /Users/niravadunuthula/repo/cs499/PoetryGen/VillanelleGen/poemsdataset/forms/abc/AbcPoemsAsSimpleAsAbcPoembyD+�nallDempsey.txt
        Illegal byte sequence
error:  cannot create /Users/niravadunuthula/repo/cs499/PoetryGen/VillanelleGen/poemsdataset/forms/abc/AbcPoemsAutographMuseAbcNameSabineB+�tzingLichtenth+�lerPoembyClintonSiegle.txt
        Illegal byte sequence
error:  cannot create /Users/niravadunuthula/repo/cs499/PoetryGen/VillanelleGen/poemsdataset/forms/anagram/AnagramPoemsLostForWordsAnAnagramConudrumTankaPoembyD+�nallDempsey.txt
        Illegal byte sequence
error:  cannot create /Users/niravadunuthula/repo/cs499/PoetryGen/VillanelleGen/poemsdataset/forms/ars-poetica/ArspoeticaPoemsArsPoeticaAtEveryTurnTodayPoembyDesmondKonZhichengMingd+�.txt
        Illegal byte sequence
error:  cannot create /Users/niravadunuthula/repo/cs499

In [11]:
poems = get_text_files(data_path, folders = ['forms','topics'])
print("There are",len(poems),"poems in the dataset")

There are 20334 poems in the dataset


In [12]:
villanelles = get_text_files(data_path+'/forms', folders = ['villanelle'])
print("There are",len(villanelles),"villanelles in the dataset")

There are 99 villanelles in the dataset


In [13]:
txt = villanelles[1].open().read(); #read the first file
print(txt)

It's as if it's you and our grandchildren I want to greet,
I hear children's voices in the early sunshine
and outside the bush-shrike's refrain sounds sweet,
the lovely refrain it does again and again repeat
and I am so glad that your love is mine.
It's as if it's you and our grandchildren I want to greet,
nearby I hear the pattering of large and small feet,
while the sheen of the sun has something almost divine
and outside the bush-shrike's refrain sounds sweet.
You and the grandchildren do my life complete
but locked-down the world is in quarantine.
It's as if it's you and our grandchildren I want to greet
but my glance does only an empty garden outside meet.
I do go back to posting some poetry
and outside the bush-shrike's refrain sounds sweet.
You are present in each and every heartbeat
and without you nothing seems to be fine.
It's as if it's you and our grandchildren I want to greet
and outside the bush-shrike's refrain sounds sweet.


In [14]:
# easy data augmentation from https://arxiv.org/abs/1901.11196
filenames = os.listdir("poemsdataset/forms/villanelle/")
for fn in filenames:
    !python eda_nlp/augment.py --input="poemsdataset/forms/villanelle/{fn}" --num_aug=5

generated augmented sentences with eda for poemsdataset/forms/villanelle/VillanellePoemsTheSameObedientOlderChildrenVillanelleSongPoemPoembyMarietaMaglas.txt to poemsdataset/forms/villanelle/VillanellePoemsTheSameObedientOlderChildrenVillanelleSongPoemPoembyMarietaMaglas.txt with num_aug=5
generated augmented sentences with eda for poemsdataset/forms/villanelle/VillanellePoemsYouArePresentInEachAndEveryHeartbeatVillanellePoembyGertStrydom.txt to poemsdataset/forms/villanelle/VillanellePoemsYouArePresentInEachAndEveryHeartbeatVillanellePoembyGertStrydom.txt with num_aug=5
generated augmented sentences with eda for poemsdataset/forms/villanelle/VillanellePoemsVillanelleOfTheTemptressPoembyJamesJoyce.txt to poemsdataset/forms/villanelle/VillanellePoemsVillanelleOfTheTemptressPoembyJamesJoyce.txt with num_aug=5
generated augmented sentences with eda for poemsdataset/forms/villanelle/VillanellePoemsICannotHelpToMoreAndMoreLoveYouVillanellePoembyGertStrydom.txt to poemsdataset/forms/villanel

In [15]:
villanelles = get_text_files(data_path+'/forms', folders = ['villanelle'])
print("There are",len(villanelles),"villanelles in the dataset")

There are 584 villanelles in the dataset


In [16]:
import numpy as np
import pandas as pd
from datasets import Dataset

def process_text(title, text):
    # strip prefix
    title = title[15:]

    # strip postfix 
    i = title.find("Poemby")
    j = title.find("VillanellePoemby")

    if (j < i and j > -1):
        i = j 
    if i != -1:
        title = title[:i]

    title = ''.join(' ' + char if char.isupper() else char.strip() for char in title).strip()
    title = "== Villanelle named '" + title + "' ==\n"

    # cut off text beyond 19 lines
    text = '\n'.join(text.split('\n')[:19])

    return title + text + '<|endoftext|>'

v = np.array(L((process_text(o.name, o.open().read()) for o in get_text_files(data_path + '/forms', folders = ['villanelle']))))  # to make things easy we will gather all texts in one numpy array
pd.DataFrame(v, columns = ["text"]).to_csv('poemsdataset/villanelles.csv')


In [13]:
!python run_clm.py \
    --model_name_or_path gpt2 \
    --train_file poemsdataset/villanelles.csv \
    --validation_split_percentage 30 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --do_train \
    --do_eval \
    --output_dir models/ft-gpt2-EDA

12/12/2022 15:41:06 - WARNING - __main__ - Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
12/12/2022 15:41:06 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half

In [43]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model = gpt2_model.to(device)

In [44]:
ft_gpt2_EDA_tokenizer = GPT2Tokenizer.from_pretrained("models/ft-gpt2-EDA")
ft_gpt2_EDA_tokenizer.pad_token = ft_gpt2_EDA_tokenizer.eos_token
ft_gpt2_EDA_model = GPT2LMHeadModel.from_pretrained("models/ft-gpt2-EDA")
ft_gpt2_EDA_model = ft_gpt2_EDA_model.to(device)

In [48]:
def gen_villanelle(input, model, tokenizer, device, k=50, p=0.95, rep_penalty=1.0):
    ids = tokenizer(input, return_tensors="pt").input_ids.to(device)

    generated_ids = model.generate(
        input_ids=ids,
        max_length=600,
        min_length=200,
        length_penalty=1,
        early_stopping=True,
        do_sample=True,
        # num_beams=4,
        # no_repeat_ngram_size=5,
        top_k=k,
        top_p=p,
        repetition_penalty=rep_penalty,
        
    )

    preds = [
        tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for g in generated_ids
    ]

    return preds[0].replace("\\n", '\n')

In [52]:
prompt = "\
== Villanelle named 'Self Help' ==\n\
What delusion are you under?\n\
The life he hid knocked you flat.\n\
You see lightning but not thunder.\n\
What God joined let none put asunder.\n\
Did God know you'd marry a rat?\n\
What delusion are you under?\n\
His online persona stunned her\n\
like when you started to chat.\n\
You see lightning but not thunder.\n\
To victors go the plunder:\n\
crown them with a baseball bat.\n\
What delusion are you under?\n\
The kind that cause blunder.\n\
Is there other kind than that?\n\
You see lightning but not thunder.\n\
the world's a wonder.\n\
keep it thrilling under your hat.\n\
What delusion are you under?\n\
You see lightning but not thunder.<|endoftext|>\n\n"

print(len(prompt))

646


In [71]:
user_input = "\
== Villanelle named 'Operating Systems' ==\n\
and Windows open the Macintosh\n\
keeping our systems running\n\
in Windows system our vandenbosch\n"
bart_input = prompt + user_input

gpt2_os = gen_villanelle(bart_input, gpt2_model, gpt2_tokenizer, device)
print("=== GPT2 ===\n" + gpt2_os[len(prompt)-15:])

ft_gpt2_EDA_os = gen_villanelle(bart_input, ft_gpt2_EDA_model, ft_gpt2_EDA_tokenizer, device)
print("=== FT GPT2 + EDA ===\n" + ft_gpt2_EDA_os[len(prompt)-15:])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GPT2 ===


== Villanelle named 'Operating Systems' ==
and Windows open the Macintosh
keeping our systems running
in Windows system our vandenbosch

becomes our main OS's the "Gemma" or "Parsnip"

the name of a Windows computer for instance is "MacOS"

when we want a new installation to install

we can use a program or GUI as the "program" so that when the

application is run or restarted we see changes to the program

we can then switch on our system.

when we want to run a command we use a text editor or a

menu.

when we want to start a command on a terminal (say, "command")

we use the program "shell"

we always have the option "stop and quit" to enable

cancel or stop the operating system.

a terminal window or menu is used by the

command to go and to change a certain status.

the shell is always run, even though it's only

run for the purpose of selecting a certain screen, the

menu is always opened, we run the program

so that we're in control of the program, all the

inform

In [72]:
user_input = "\
== Villanelle named 'Evening in a Forest Glade' ==\n\
decided to enjoy a lonely trail and glade\n\
accustomed to the darkness light and foggy\n\
surely dinner is already being made\n"
bart_input = prompt + user_input

gpt2_forest = gen_villanelle(bart_input, gpt2_model, gpt2_tokenizer, device)
print("=== GPT2 ===\n" + gpt2_forest[len(prompt)-15:])

ft_gpt2_EDA_forest = gen_villanelle(bart_input, ft_gpt2_EDA_model, ft_gpt2_EDA_tokenizer, device)
print("=== FT GPT2 + EDA ===\n" + ft_gpt2_EDA_forest[len(prompt)-15:])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GPT2 ===


== Villanelle named 'Evening in a Forest Glade' ==
decided to enjoy a lonely trail and glade
accustomed to the darkness light and foggy
surely dinner is already being made

all will meet soon

but now the light

will not fade from his eyes as fast as the light fades

and with his best friend he will not fall

when he returns, to enjoy the journey

from forest to glade

to camp to the cave and back again

until the daylight dawns

He will be in one camp

but it will be dark on the other side

The camp will not become quiet anymore

when the dusk dawns,

the dark will not fade

but it will continue dark

while there will be a trail to the cave

for many nights

in the middle of the night there will be no sight

to see you and your friends

without darkness

at night no sight

to see that which they see

to see

not seen or that they can see

the darkness in your heart.

The night is far past, no more

you can see,

but with an inch as tall as a half a mile

you know, even 

In [73]:
user_input = "\
== Villanelle named 'A Pokemon Adventure' ==\n\
father is amazing nothing like the game\n\
we collected some and started aiming\n\
gadgets are in helping us improve the aim\n"
bart_input = prompt + user_input

gpt2_pokemon = gen_villanelle(bart_input, gpt2_model, gpt2_tokenizer, device)
print("=== GPT2 ===\n" + gpt2_pokemon[len(prompt)-15:])

ft_gpt2_EDA_pokemon = gen_villanelle(bart_input, ft_gpt2_EDA_model, ft_gpt2_EDA_tokenizer, device)
print("=== FT GPT2 + EDA ===\n" + ft_gpt2_EDA_pokemon[len(prompt)-15:])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GPT2 ===


== Villanelle named 'A Pokemon Adventure' ==
father is amazing nothing like the game
we collected some and started aiming
gadgets are in helping us improve the aim

you do not like the graphics

it has a ton of bugs and glitches but its always fun

you cant beat it

no

we will make another game about pokemon on steam!


(http://steamcommunity.com/profiles/765611982216123580)
=== FT GPT2 + EDA ===
== Villanelle named 'A Pokemon Adventure' ==
father is amazing nothing like the game
we collected some and started aiming
gadgets are in helping us improve the aim
the school runs a free play on top of pokemon
it was a challenge to learn how pokemon works
the school goes free play free play
go down to do the test where it comes to life
a pokemon can never tell you what a big bad bballer will throw
the pokemon has the look and feel of a human
my dream comes true
my dream comes true
we have to learn something to be nice
a pokemon called 'A Pokemon Adventure'
my wish never comes t

In [79]:
user_input = "\
== Villanelle named 'A Blooming Spring Field' ==\n\
blossoms bursting open me and blooming\n\
popping open blossoms rather quickly\n\
Sorry It's the carpet I'm assuming\n"
bart_input = prompt + user_input

gpt2_bloom = gen_villanelle(bart_input, gpt2_model, gpt2_tokenizer, device)
print("=== GPT2 ===\n" + gpt2_bloom[len(prompt)-15:])

ft_gpt2_EDA_bloom = gen_villanelle(bart_input, ft_gpt2_EDA_model, ft_gpt2_EDA_tokenizer, device)
print("=== FT GPT2 + EDA ===\n" + ft_gpt2_EDA_bloom[len(prompt)-15:])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GPT2 ===


== Villanelle named 'A Blooming Spring Field' ==
blossoms bursting open me and blooming
popping open blossoms rather quickly
Sorry It's the carpet I'm assuming
So we've got this whole whole thing in hand
And as I look and I see
Oh, the sweet bloom that makes our whole field glow
I don't know when or where
Blossom is going to bloom but, the flower will bloom
All right, and when the blooms show
We shall see our new bloom in bloom so loud
And I'll do it again so loud
Blossom will bloom in the morning, and bloom for a while
After all, if it does show in bloom.
So let's go.

Oh, it's you

There's nothing wrong with being a little

Blossom is one thing but love and flowers are

In this whole field that we grow so

Don't be such a bunchie

And do everything possible for yourself

I'll make it even more big

The blooms of my field and my garden

And make everything bloom again

So I'm happy there I am

This field is a little too big

I'm so happy there I am

And then I want a li

In [77]:

for title in ["Operating Systems", "Evening in a Forest Glade", "A Pokemon Adventure", "A Blooming Spring Field"]:
    user_input = f"======= Villanelle named '{title}' =======\n"
    bart_input = prompt + user_input
    print(user_input)

    out = gen_villanelle(bart_input, gpt2_model, gpt2_tokenizer, device)
    print("=== GPT2 ===\n" + out[len(prompt)-14:])

    out = gen_villanelle(bart_input, ft_gpt2_EDA_model, ft_gpt2_EDA_tokenizer, device)
    print("=== FT GPT2 + EDA ===\n" + out[len(prompt)-14:])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


======= Villanelle named 'Operating Systems' =======



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GPT2 ===

======= Villanelle named 'Operating Systems' =======

======= Villanelle name will be renamed to V6.1.


v6:

0.01/V7:

Version 1.5

- Improved performance

- Improved performance with more options

- Made some minor tweaks to all the menus.

- Added support for 2nd party game engine.

- Added a number of minor tweaks and improvements for v6

- Added the 'Open File' button (default is 'Open')

- Added a number of minor tweaks and improvements for v6

- Added the 'Create More' button (default is 'Create More')

- Added the 'Close File' button (default is 'Close')

- Added a number of minor tweaks and improvements for v6

- Fixed a few bugs and optimisations

- Minor tweaks for v6

- Fixed a few small typos.



v6:

- Improved performance with more options

- Improved performance with more options - Fixed some bugs

- Minor tweaks for v6 - Tweaked the default option in the 'Open Files' button.

- Fixed a bug in the 'New Files' button to allow other users to choose.

- Fixed

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

=== GPT2 ===

======= Villanelle named 'A Pokemon Adventure' =======

'A Pokemon Adventure' is a long term project (more than 4 years) and involves some fun. It is about a woman who wants to meet up with one of her Pokemon in time to catch a rare Pokemon with some mysterious power in order to make her a god of Pokemon with her.

I will not be working on the game anymore because it is a short-term project and it can take a little longer. The main character is not going to be able to run and interact with Pokemon on the Nintendo Switch, so the project will likely end sooner. I would welcome feedback. However I also want to mention that this game is far more about the experience and not about getting into that game alone.

I did not design the game to be a Pokemon game like 'Pokemon', but rather one where you have the chance to encounter different types of Pokemon. There is no real control system to the controls you need to do in order to progress. There are different types of Pokemon, an

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== FT GPT2 + EDA ===
====== Villanelle named 'A Pokemon Adventure' =======
A Pokemon Adventure is a journey through the world of Velociraptor. Along the way you'll find many treasures.
As you head out to the vast wilderness and explore you'll meet new Pokemon along the way.
Your favourite Pokemon roam around the wild areas and on the plains.
======= Villanelle named 'A Blooming Spring Field' =======



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GPT2 ===

======= Villanelle named 'A Blooming Spring Field' =======

======= Villanelle named 'Carpetball' =======

======= Villanelle named 'Dawnstar' =======

======= Villanelle named 'Oblivion Hill' =======

======= Villanelle named 'Cedar Hill' =======

======= Villanelle named 'Oblivion Hill' =======

======= Villanelle named 'Woodlawn Woods' =======

======= Villanelle named 'Waterfalls' =======

======= Villanelle named 'Dawnstone' =======

======= Villanelle named 'Woodland Village' =======

======= Villanelle named 'Jamaica Creek' =======

======= Villanelle named 'Jamaica Creek' =======

======= Villanelle named 'Wake Hill' =======

======= Villanelle named 'Lombard Hill' =======

======= Villanelle named 'Twilight Hill' =======

======= Villanelle named 'Tropical Ridge' =======

======= Villanelle named 'Lincoln Forest' =======

======= Villanelle named 'Lincoln Creek' =======

======= Villanelle named 'Waterfalls' =======

======= Villanelle named 'Twilight' =======

=

In [6]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset, random_split

In [81]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

class VillanelleDataset(Dataset):
    def __init__(self, path, tokenizer):
        villanelles = L(o.open().read() for o in get_text_files(path, folders = ['villanelle'])) # to make things easy we will gather all texts in one numpy array
        self.all_villanelles = flatten(villanelles)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.all_villanelles)

    def __getitem__(self, idx):
        tokenized = self.tokenizer(self.all_villanelles[idx], padding='max_length', return_tensors="pt")
        tokenized["labels"] = tokenized["input_ids"].clone()
        # print(tokenized.items())
        return tokenized

In [82]:
villanelle_dataset = VillanelleDataset(data_path + '/forms', tokenizer)

valid_split = 0.3 # 70/30 split
dataset_size = len(villanelle_dataset)
valid_size = int(valid_split * dataset_size)
train_size = dataset_size - valid_size
train_dataset, valid_dataset = random_split(villanelle_dataset,
                                               [train_size, valid_size])

In [84]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [85]:
trainer.train()

/Users/niravadunuthula/repo/cs499/CSCI499_NaturalLanguageforInteractiveAI/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 70
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 27
  Number of trainable parameters = 124439808
















































































***** Running Evaluation *****
  Num examples = 29
  Batch size = 8




































                                        
                                             



                          







                      





{'eval_loss': 0.6599388122558594, 'eval_runtime': 149.3276, 'eval_samples_per_second': 0.194, 'eval_steps_per_second': 0.027, 'epoch': 1.0}










































































***** Running Evaluation *****
  Num examples = 29
  Batch size = 8




































                                        
                                             



                            







                       









               

                                           


                              




                         






                





                            








  0%|          | 0/27 [2:55:04<?, ?it/s]          























































{'eval_loss': 0.6431915163993835, 'eval_runtime': 149.9395, 'eval_samples_per_second': 0.193, 'eval_steps_per_second': 0.027, 'epoch': 2.0}










































































***** Running Evaluation *****
  Num examples = 29
  Batch size = 8




































                                        
                                             



                            







                         

                                           


                              




                         






                  





                              









               








  0%|          | 0/27 [3:29:26<?, ?it/s]          























































Training completed. Do not forget to share your model on huggingface.co/models =)


                                        
                                             



                            







                         

                                            


                              




                         






             

{'eval_loss': 0.6397735476493835, 'eval_runtime': 160.2, 'eval_samples_per_second': 0.181, 'eval_steps_per_second': 0.025, 'epoch': 3.0}
{'train_runtime': 6017.7894, 'train_samples_per_second': 0.035, 'train_steps_per_second': 0.004, 'train_loss': 0.740012698703342, 'epoch': 3.0}


100%|██████████| 27/27 [1:40:17<00:00, 222.86s/it]


TrainOutput(global_step=27, training_loss=0.740012698703342, metrics={'train_runtime': 6017.7894, 'train_samples_per_second': 0.035, 'train_steps_per_second': 0.004, 'train_loss': 0.740012698703342, 'epoch': 3.0})